Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 4 </h2>
    <h1> Clustering, Asociaciones y Redes Bayesianas </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Primer Semestre 2025<br>    
        Fecha de entrega: 23 de mayo
    </p>
    <br>
</center>

<br>

---

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

---

## Librerías

A continuación se encuentran las librerías necesarias para elaborar la tarea. Recuerda ejecutar la celda antes de comenzar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity


import umap.umap_ as UMAP

from pgmpy.inference import VariableElimination
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch, BDs

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#Librerías aparte
from sklearn import cluster
from collections import Counter



## Contexto

Un empresario multimillonario acaba de comprar un club de fútbol de las 5 mejores ligas del mundo y te ha contactado a ti como experto en Mineria de Datos para que lo ayudes a saber como llevar a su nuevo equipo a los primeros lugares. Para conseguir lo anterior te ha entregado datos de la temporada 2015-2016 de la Bundelisga (Alemania), La Liga (España), Ligue 1 (Francia), Premier League (Inglaterra) y Serie A (Italia) con las estadísticas de los equipos que participaron en esa temporada en sus respectivas ligas para poder encontrar estilos de juegos que le puedan servir para replicar en su nuevo equipo.

## 1. Carga y Preprocesamiento de Datos (5 Pts.)

### 1.1 Cargar Datos

Cargue el dataset `teams.csv`. Indique la cantidad de filas y columnas del conjunto de datos. ¿Existen datos nulos? De ser así, mencione el número de casos por variable y luego elimine todas filas que tengan a lo menos un valor nulo.

In [ ]:
df_teams = pd.read_csv("teams.csv")
df_teams = df_teams.drop(["Unnamed: 0"], axis = 1)

In [ ]:
df_teams.info()

El dataset contiene 77 filas (Sin contar el header) y 27 columnas 

Además aparentemente (Al menos según pandas y posterior indagación dentro del archivo .csv) no existen datos nulos

### 1.2 Descripción del dataset (1 Pts.)

Describa brevemente cuantas columnas tiene el dataset y defina que significa cada columna del dataset. Para lo anterior te puedes apoyar de las siguientes páginas webs:

- [Definición Métricas Opta](https://theanalyst.com/2024/07/opta-football-stats-definitions)
- [Explicación xG Opta](https://theanalyst.com/2023/08/what-is-expected-goals-xg)
- [Definición Métricas StatsPerform](https://www.statsperform.com/opta-event-definitions/)
- [Glosario WhoScored](https://www.whoscored.com/glossary)

In [ ]:
df_teams

In [ ]:
df_teams.info()

Las columnas del dataset y lo que representa cada una es:
1) team: Representa el equipo, veo que este tiene un valor unico por columna
2) total_shots: Representa el número de veces que un equipo ha hecho un tiro
3) onTarget: según la definición de métricas Opta, esta columna representaría el número de veces que se ha pateado la pelota hacia portería, incluyendo las veces que se ha hecho un gol (o score) como aquellas en las que el arquero lo ha evitado
4) goals: Número de veces que un equipo ha hecho un gol
5) xG_total: si xG representa la probabilidad de que un tiro resulte en gol, xG_total representa la suma de todos los xG de los tiros de un equipo
6) Shot_to_goal_ratio: Sería la división entre el número de goles partido por el número total_shots de un equipo
7) Shot_onTarget_to_goal_ratio: Sería la división entre el número de goles partido por el número onTarget de un equipo
8) xG/shot: Corresponde a la fracción de la probabilidad de que un tiro resulte en gol partido por el número de tiros
9) recoveries: Número de veces en la que un equipo ha recuperado la pelota en situaciones en las que ninguno de los equipos la tiene o cuando la pelota ha sido jugada directamente hacia el por un equipo rival, así asegurando la posesión de la pelota para el equipo
10) recoveries_f3: Es lo mismo que recoveries pero hecha en la zona F3 del campo de futbol, siendo F3 la zona defensiva del equipo rival (el tercio final del campo contrario)
11) duels_won: Esta métrica aplicada a un equipo representaría el número de duelos individuales entre jugadores del equipo contra jugadores de equipos rivales que los jugadores del equipo han ganado. Siendo un duelo una disputa directa por el balón
12) fouls_committed: Número de veces que jugadores del equipo han infringido reglas del juego.
13) shots_against: Número de intentos de gol que el equipo rival ha realizado
14) shots_onTarget_against: Lo mismo que shots_against solo que estos se refieren a tiros que iban dirigidos directos a portería y cuya línea de defensa es solo el arquero
15) xGA: Es lo mismo que xG pero tomando solo los tiros hechos por el equipo rival
16) goals_against: Es el número de goles que les han hecho al equipo
17) clean_sheets: Número de partidos en los que el equipo no ha recibido goles.
18) ppda: Passes Per Defensive Action. Mide la intensidad de la presión de un equipo. Un PPDA bajo indica una presión más alta.
19) total_passes: El número total de pases realizados por el equipo.
20) f3_passes: Pases realizados en el último tercio del campo (zona f3).
21) short_passes: Pases de menos de 30 metros de distancia.
22) long_passes: Pases de 30 metros o más de distancia.
23) average_pass_length: La distancia promedio de los pases del equipo.
24) crosses: Se refiere a los pases que se envían desde las bandas del campo hacia el área de penalti, generalmente con la intención de que un compañero remate a portería. Son una métrica clave para evaluar la creación de oportunidades de gol desde las alas del terreno de juego.
25) ranking: Indica la posición actual de un equipo dentro de una liga, torneo o competición en particular. Este puesto se determina en función de un sistema de puntuación (victoria, empate, derrota) o de otros criterios de rendimiento establecidos por la competición.
26) points_per_match: Es una métrica que calcula el promedio de puntos que un equipo ha obtenido en cada partido disputado a lo largo de una temporada o competición. Se obtiene dividiendo el total de puntos acumulados por el número de partidos jugados. Es un buen indicador de la consistencia del rendimiento de un equipo.
27) groups: Esta columna representa el grupo al que pertenece el equipo

### 1.3 Normalización (1 Pts.)

Realice el proceso de normalización sobre la matriz de variables explicativas utilizando el método StandardScaler. A partir de aquí, tiene que usar esta matriz para los experimentos. Para lo anterior, elimine las columnas `team`, `ranking`, `points_per_match` y `groups`, pero mantenga una copia del dataframe original, ya que lo necesitaras en secciones posteriores.

In [ ]:
df_teams_copy = df_teams.copy()
df_teams_copy= df_teams_copy.drop(["team", "ranking", "points_per_match", "groups"], axis = 1)

df_teams_copy.info()

In [ ]:
df_teams_copy.info()

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df_teams_copy)
X

### 1.4 Matriz de correlación (2 Pts.)

A partir del dataframe normalizado en la sección anterior construya la matriz de correlación. ¿Que variables tienen alta correlación ($> 0.8$)?

In [ ]:
df_teams_copy.info()

In [ ]:
nombres_cols = df_teams_copy.columns
X_normalizado = pd.DataFrame(X, columns=nombres_cols)
correlation_matrix = X_normalizado.corr()

In [ ]:
high_corr_pairs = list()
for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        correlation_value = correlation_matrix.iloc[i, j]

        if abs(correlation_value) > 0.8:
            high_corr_pairs.append((col1, col2, correlation_value))

if high_corr_pairs:
    for pair in high_corr_pairs:
        print(f"  {pair[0]} - {pair[1]}: {pair[2]:.4f}")

## 2. Clustering (15 Pts.)

### 2.1 K-Means (5 Pts.)

#### 2.1.1 Explicación teórica (2 Pts.)

Investigue sobre `K-means` y entregue una explicación de como funciona. `No es necesario una explicación matemática`. Se espera que tu explicación contenga la respuesta a las siguientes preguntas:
1. ¿Qué tipo de algoritmo es?
2. ¿Para qué sirve?
3. ¿Cómo funciona?
4. ¿Qué es un centroide?
5. ¿Que es la inercia en este contexto?

**❗IMPORTANTE❗** No utilices ChatGPT u otro modelo de lenguaje para responder esta pregunta. Será evaluado que cites al menos 1 recurso bibliográfico que hayas utilizado. Cualquier información que no tenga una referencia clara no se le dará puntaje.

El algoritmo K-means es un algoritmo de clusterización mediante el cual se clasifica un grupo de datos en k categorías o clusters. Generalmente es usado para etiquetar datos no etiquetados. El procedimiento es el siguiente:
1) Se empieza aleatoriamente eligiendo k puntos de los datos que harán de centroides inicialmente, nostros querremos que estos puntos sean el punto central o promedio de cada cluster más adelante
2) "Asignar cada punto del conjunto de datos al cluster cuyo centroide esté más cerca" (fuente: https://www.iebschool.com/hub/algoritmo-k-means-que-es-y-como-funciona-big-data/). La metrica usada para medir que tan cerca está un punto de un centroide generalmente es la distancia euclideana (Fuente: https://www.ibm.com/mx-es/think/topics/k-means-clustering)
3) Una vez agrupados los datos en distintos clusters, en cada cluster se calcula la posición media de los datos del cluster, para después actualizar la posición del centroide para que siga siendo el "centro" o que esté en la posición media del cluster
4) Repetimos los pasos 2 y 3 hasta que la posición del centroide ya no cambie o se alcance el número máximo de iteraciones (https://www.iebschool.com/hub/algoritmo-k-means-que-es-y-como-funciona-big-data/), o "hasta la variación de la inercia no sea significativa" (fuente https://adrianchust.com/algoritmo-kmeans/). ya que según la misma página de donde obtuve la información anterior, la inercia es "... el medidor que nos indica cómo de bien se han ajustado los datos" y esta se calcula "midiendo la distancia entre un punto de datos y su centroide" (fuente: https://www.ibm.com/mx-es/think/topics/k-means-clustering#:~:text=La%20inercia%20se%20calcula%20midiendo%20la%20distancia%20entre,suma%20o%20valor%20inercial%20es%20la%20distancia%20intracluster.)

Fuentes: 
- https://www.iebschool.com/hub/algoritmo-k-means-que-es-y-como-funciona-big-data/
- https://www.ibm.com/mx-es/think/topics/k-means-clustering

#### 2.1.2 Número de clusters (2 Pts.)

La libreria [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) tiene implementado este algoritmo. Grafique los valores de *inertia* y silhoutte score para un rango de 2 a 10 clusters. Determine el número óptimo de clusters.

In [ ]:
inertia_values = []
silhouette_scores = []
cluster_range = range(2, 11)

for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=40, n_init=10)
    kmeans.fit(X_normalizado)
    inertia_values.append(kmeans.inertia_)
    
    if n_clusters > 1:
        silhouette_avg = silhouette_score(X_normalizado, kmeans.labels_)
        silhouette_scores.append(silhouette_avg)
    else:
        silhouette_scores.append(0)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(cluster_range, inertia_values, marker='o')
plt.title('Inertia v/s nro clusters')
plt.xlabel('Número de Clusters')
plt.ylabel('Inertia')
plt.xticks(cluster_range)
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(cluster_range, silhouette_scores, marker='o', color='red')
plt.title('Silhouette Score')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')
plt.xticks(cluster_range)
plt.grid(True)

plt.tight_layout()
plt.show()


Para decidir cuál nro de clusters elegir, decido aquel que sea beneficioso para ambos, es decir, aquel que me capture el mayor nivel de variación de inercia a su izquierda y que tenga preferiblemente un valor de silhouette score lo más alto posible. Por lo que decido tomar a la vista n_clusters = 4

#### 2.1.3 Ejecutar K-Means (1 Pts.)

**Ejecute K-Means con un número de clusters igual a 4.** Una vez obtenidos los clusters imprima las posiciones en que terminaron esos equipos:

- ¿Existe alguna relación entre los clusters obtenidos y la posición que obtuvieron los equipos de ese cluster respectivo?
- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

Finalmente, calcule *silhouette score* con los valores escogidos.

In [ ]:
X_data_for_kmeans = X_normalizado.values
n_clusters_optimal = 4

kmeans = KMeans(n_clusters=n_clusters_optimal, random_state=40, n_init=10)
kmeans.fit(X_data_for_kmeans)

labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Utilizo pca ya que X_normalizado tiene más de 2 columans
pca = PCA(n_components=2, random_state=40)

# Aplico PCA a mis datos
X_pca = pca.fit_transform(X_data_for_kmeans)

centroids_pca = pca.transform(centroids)

sns.set_style("whitegrid")
plt.figure(figsize=(10, 8))

scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1],
                      c=labels, cmap='viridis', s=50, alpha=0.8, edgecolors='w')

plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1],
            c='red', marker='X', s=200, label='Centroides', edgecolors='black')

plt.title(f'Clusters de K-Means (K={n_clusters_optimal}) - Visualización con PCA')
plt.xlabel(f'Componente Principal 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
plt.ylabel(f'Componente Principal 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')

plt.legend()
plt.colorbar(scatter, ticks=range(n_clusters_optimal), label='ID de Cluster')

plt.show()


In [ ]:
X_normalizado

In [ ]:
df_teams['cluster_label'] = kmeans.fit_predict(X_data_for_kmeans)

print("DataFrame con etiquetas de cluster:")
print(df_teams[['team', 'ranking', 'cluster_label']])

print("\n--- Posiciones de los equipos por Cluster ---")
for cluster_id in sorted(df_teams['cluster_label'].unique()):
    cluster_teams = df_teams[df_teams['cluster_label'] == cluster_id]
    print(f"\nCluster {cluster_id}:")
    print(cluster_teams[['team', 'ranking']].to_string(index=False)) 

Puedo ver entonces que no en todos los clusters hay una relación clara entre el id del cluster y el ranking o posición en la que se encuentra el equipo, ya que solo para el cluster 2 se podría decir que k-means obtiene resultados donde la mayoría de los datos agrupados en cluster 2 tienen un ranking que coincide que el id del cluster

- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

In [ ]:
# Cluster 0
cluster_0 = df_teams[df_teams['cluster_label'] == 0]

cluster_0[["ranking", "cluster_label"]]


In [ ]:
# cluster 1 
cluster_1 = df_teams[df_teams['cluster_label'] == 1]
cluster_1[["ranking", "cluster_label"]]


In [ ]:
# Cluster 2
df_teams[df_teams['cluster_label'] == 2]

cluster_2[["ranking", "cluster_label"]]


In [ ]:
# Cluster 3
cluster_3 = df_teams[df_teams['cluster_label'] == 3]
cluster_3[["ranking", "cluster_label"]]


Como podemos volver a ver, No hay una relación clara entre el id del cluster y el ranking de los equipos que son agrupados dentro de él. Por lo que voy a evaluar otras características

In [ ]:
pd.set_option('display.max_columns', None)

cluster_0.describe()

In [ ]:
cluster_1.describe()

In [ ]:
cluster_2.describe()

In [ ]:
cluster_3.describe()

cluster_0 según mi análisis, es el cluster que agrupa a los mejores equipos del dataset, ya que contiene el mayor valor promedio de goals (Sin considerar a cluster_2 ya que este tiene muchos menos datos en consideración), además que cluster_0 posee el mayor valor promedio de points_per_match, que según afirmé basado en Opta que este es un muy buen indicador del rendimiento del equipo. Finalmente también contiene el mayor promedio de ranking

Cluster_1 este cluster contiene equipos con un promedio menor que el de cluster_0 y mayor que el de cluster_3, lo mismo pasa con su número de goles promedio, por lo que quizás este pueda describirse como el cluster de los segundo mejores equipos del dataset. Sin embargo contiene el mayor número de faltas promedio, lo que podría clasificarlo como un grupo de jugadores agresivos. Además que contiene el segundo mayor valor promedio en ranking

En cuanto a cluster_2, es un grupo que al tener la mitad de los datos de cluster_0, podría confundirse con el cluster de los equipos de elite. Sin embargo dada la poca información que se tiene respecto a este en comparación a los demás clusters, no es comparable. Sin embargo, Contiene grupos en general muy buenos, con un Shot_onTarget_to_goal_ratio promedio mayor que el de los demás clusters

Cluster_3 Finalmente es el cluster con los tercero mejores equipos del dataset, con el menor valor promedio de ranking y el mayor valor de faltas cometidas promedio en comparación a cluster_0 y cluster_1. 

### 2.2 DBSCAN (5 Pts.)

Usando *k-dist plot* para MinPts=3, encuentre que rango de EPS deberiamos evaluar. Una vez obtenidos los clusters realice el mismo análisis de la sección anterior:

- ¿Existe alguna relación entre los clusters obtenidos y la posición que obtuvieron los equipos de ese cluster respectivo?
- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

Finalmente, calcule *silhouette score* con los valores escogidos

In [ ]:
MinPts = 3 
k = MinPts - 1

neigh = NearestNeighbors(n_neighbors=MinPts, metric='euclidean')
nbrs = neigh.fit(X_normalizado)
distances, indices = nbrs.kneighbors(X_normalizado)
k_distances = np.sort(distances[:, k], axis=0)

plt.figure(figsize=(10, 6))
plt.plot(k_distances)
plt.xlabel("Índice de puntos (ordenado por distancia k)")
plt.ylabel(f"Distancia al {MinPts-1}-ésimo vecino más cercano")
plt.title(f"K-distance plot para MinPts = {MinPts}")
plt.grid(True)
plt.show()


Entonces si debemos clusterizar muchos datos, debemos usar un EPS ~ [3.5, 5.5]

In [ ]:
dbscan = cluster.DBSCAN(eps=3.5, min_samples=MinPts)
clusters = dbscan.fit_predict(X_normalizado)

df_teams['DBSCAN_Cluster'] = clusters
#cluster_0['DBSCAN_Cluster'] = clusters

cluster_counts = Counter(clusters)
print("Frecuencia de etiquetas de cluster:", cluster_counts)

Entonces veo que DBSCAN ha identificado 2 clusters y outliers -1

In [ ]:
y_pred = clusters.astype(int)

def colour(x):
    return plt.cm.Dark2(x)

colors = list(map(colour,y_pred))

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_normalizado)

plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters, palette='viridis', s=50, alpha=0.7)
plt.title('Clústeres de DBSCAN (Visualización con PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Clúster')
plt.grid(True)
plt.show()

In [ ]:
df_teams

In [ ]:
# Cluster 0
cluster_0 = df_teams[df_teams['DBSCAN_Cluster'] == 0]
cluster_0[["ranking", "DBSCAN_Cluster"]]


In [ ]:
# Cluster 1
cluster_1 = df_teams[df_teams['DBSCAN_Cluster'] == 1]
cluster_1[["ranking", "DBSCAN_Cluster"]]


In [ ]:
# Cluster outliers
cluster_01 = df_teams[df_teams['DBSCAN_Cluster'] == -1]
cluster_01[["ranking", "DBSCAN_Cluster"]]

In [ ]:
cluster_0.describe()

In [ ]:
cluster_1.describe()

In [ ]:
cluster_01.describe()

In [ ]:
for elemento in cluster_0.describe().mean():
    print(elemento/17) # Divido por el número de datos para poder comparar

In [ ]:
for elemento in cluster_1.describe().mean():
    print(elemento/37)

In [ ]:
for elemento in cluster_01.describe().mean():
    print(elemento/24)

Es así que puedo ver en base al análisis que cluster_0 contiene equipos que son ligeramente mejor que los de cluster_1, al igual que points_per_match y ranking son ligeramente diferentes entre ambos clusters. Quizás cluster_01 correspondiente a los outliers sea ligeramente peor en rendimiento que cluster_0 según points_per_match.

### 2.3 GMM (5 Pts.)

#### 2.3.1 Explicación AIC (2 Pts.)

Investigue sobre las métricas `AIC`, y conteste las siguientes preguntas:

- ¿Qué es la métrica `AIC`?
- ¿Qué penaliza esta métrica?
- ¿Qué casos de usos tiene esta métrica?
- Para el caso de GMM, ¿se busca maximizar o minimizar esta métrica? ¿Por qué?

**❗IMPORTANTE❗** No utilices ChatGPT u otro modelo de lenguaje para responder esta pregunta. Será evaluado que cites al menos 1 recurso bibliográfico que hayas utilizado. Cualquier información que no tenga una referencia clara no se le dará puntaje.

#### 2.3.2 Obtener AIC (1 Pts.)

Determine el número óptimo de componentes para el modelo Gaussian Mixture Model utilizando la métrica AIC. Considere un rango de número de componentes entre 1 y 40.

Luego, grafique la curva entre número de componentes y la métrica AIC

#### 2.3.3 Implementación GMM (2 Pts.)

Escoja el número de componentes que minimiza AIC para entrenar GMM. Una vez obtenidos los clusters realice el mismo análisis de la sección anterior:

- ¿Existe alguna relación entre los clusters obtenidos y la posición que obtuvieron los equipos de ese cluster respectivo?
- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

Finalmente, calcule *silhouette score* con los valores escogidos.

## 3. Visualización (10 Pts.)

En esta sección se espera que grafiques los datos coloreados según su cluster utilizando PCA, t-SNE y UMAP.

### 3.1 PCA n=2 (3 Pts.)

Realiza una gráfica de PCA con 2 componentes coloreando cada dato según su cluster para los resultados obtenidos con K-Means, DBSCAN y GMM.

### 3.2 t-SNE (2 Pts)

Realiza una gráfica de t-SNE con 2 componentes coloreando cada dato según su cluster para los resultados obtenidos con K-Means, DBSCAN y GMM.

### 3.3 UMAP (3 Pts.)

Realiza una gráfica de UMAP con 2 componentes coloreando cada dato según su cluster para los resultados obtenidos con K-Means, DBSCAN y GMM.

### 3.4 Conclusiones (2 Pts.)

En base a lo observado en los gráficos obtenidos en la sección anterior conteste:

- ¿Que método de reducción de dimensionalidad permite ver de mejor manera la agrupación de equipos?
- ¿Observaste alguna relación entre los silhoutte score obtenidos y como se visualizaron los clusters en los gráficos anteriores? ¿Qué método de clusterización se comportó mejor al graficarlo?
- ¿Existió algún cluster que se mantuvo entre las distintas técnicas de clusterización?

## 4. Redes Bayesianas (15 Pts.)

A continuación debes realizar un análisis de redes bayesianas sobre el dataset, con la finalidad de poder inferir la probabilidad de que un equipo pertenzca a cierto grupo o no, dadas las características que se presentan en el dataset.

### 4.1 Explicación Teórica (5 Pts.)

Investiga sobre `Redes Bayesianas` y entrega una explicación de cómo funciona. `No es necesaria una explicación matemática`, el objetivo es que puedas tomar lo que aprendiste en clases o buscando en internet, y logres exponerlo de manera sintetizada para demostrar tu aprendizaje. Se espera que tu explicación contenga la respuesta a las siguientes preguntas:
1. ¿Qué es?
2. ¿Para qué sirve?
3. ¿Cómo se construye?
4. ¿Qué es un nodo y una arista en una red bayesiana?
5. ¿Qué es un CPD?

**❗IMPORTANTE❗** No utilices ChatGPT u otro modelo de lenguaje para responder esta pregunta. Será evaluado que cites al menos 1 recurso bibliográfico que hayas utilizado. Cualquier información que no tenga una referencia clara no se le dará puntaje.

### 4.2 Aprender la estructura de la red bayesiana (3 Pts.)

Del DataFrame original, elimine la columna `team`.
Luego, utilice la librería pgmpy en Python para aplicar el algoritmo Hill Climb Search con el criterio de puntuación BDs y encontrar la estructura óptima de una red bayesiana. Puedes guiarte de las actividades vistas en clases.

(Sugerencia: la clase `BDs` se importa de forma simiar a `BIC`, es decir, `from pgmpy.estimators import BDs`)

### 4.3 Encontrar la estructura óptima (2 Pts.)

En esta sección, debes entrenar la red bayesiana con los datos de entrenamiento usando la estructura óptima encontrada en la sección anterior, ajustando los parámetros usando priors de Dirichlet con cuentas uniformes. Luego, debes imprimir las probabilidades condicionales de la red bayesiana.

### 4.4 Inferencia sobre los datos (5 Pts.)

Finalmente, debes realizar inferencia sobre la red bayesiana. Para esto, se te pide que calcules la probabilidad de que un equipo pertenezca al grupo `groups` dadas las siguientes características

- Equipo 1: 'fouls_committed': 706, 'duels_won': 433
- Equipo 2: 'total_shots': 439, 'total_passes': 15919
- Equipo 3: 'goals': 80, 'goals_against': 51, 'onTarget': 137

¿Que limitaciones cree que tiene este modelo para el problema?

## 5. Asociaciones (15 Pts.)

Debido a tu gran desempeño en tu labor anterior detectando los distintos tipos de equipos en las 5 grandes ligas de fútbol, el empresario multimillonario te quiere encargar una nueva misión. Se ha enterado que el último campéon de la Eurocopa ha sido España, por lo que quiere fichar jugadores de esta selección que se relacionen bien entre ellos para poder ficharlos. Para conseguir lo anterior, te ha entregado un dataset con las secuencias de pases de los jugadores de la Selección Española durante sus partidos en la Eurocopa 2024 y tu labor será encontrar los jugadores que mejor se asocian.

Una secuencia de pases se entiende como todos los jugadores que se dieron pases entre ellos antes de que el equipo haya perdido el balón o el juego se haya detenido por algún motivo. El dataset contiene lineas con los nombres de los jugadores que participaron en una secuencia de pases separados por ;.

### 5.1 Cargar Datos (2 Pts.)

Carga el dataset `passes.csv` y dejando los datos como una lista de listas.

### 5.2 Mostrar frecuencia (3 Pts.)

A continuación debes mostrar la frecuencia de participación de los jugadores en las secuencias de pases ordenadas de manera descendente.

### 5.3 Transformar datos (3 Pts)

Utiliza `TransactionEncoder()` para codificar las secuencias de pases en vectores.

### 5.4 Matriz de co-ocurrencia (2 Pts.)

Genera la matriz de co-ocurrencia para observar con qué frecuencia los jugadores participan entre sí en las secuencias de pases. En base a la matriz contesta las siguientes preguntas:

- ¿Qué jugadores se observa que estan relacionados?
- Revisando las formaciones de España durante la Eurocopa, ¿se observa alguna relación entre las posiciones en el campo y los jugadores con mayor relación entre ellos?

[España v/s Croacia](https://www.sofascore.com/football/match/croatia-spain/YTbspUb#id:11873907)

[España v/s Italia](https://www.sofascore.com/football/match/italy-spain/YTbshUb#id:11873901)

[España v/s Albania](https://www.sofascore.com/football/match/spain-albania/PTbsYTb#id:11873904)

[España v/s Georgia](https://www.sofascore.com/football/match/georgia-spain/YTbsnVb#id:11874031)

[España v/s Alemania](https://www.sofascore.com/football/match/germany-spain/YTbslUb#id:11874027)

[España v/s Francia](https://www.sofascore.com/football/match/germany-spain/YTbslUb#id:11874027)

[España v/s Inglaterra](https://www.sofascore.com/football/match/england-spain/YTbsnUb#id:11874023)

### 5.5 Algoritmo a-priori (5 Pts.)

Aplica el algoritmo Apriori con min_support=0.1 y analiza los resultados obtenidos.

A continuación, genera las reglas de asociación utilizando association_rules, con un umbral min_threshold=0.3, y en base a los resultados obtenidos conteste las siguientes preguntas:

- ¿Qué par de jugadores recomendaría que se ficharan juntos?
- ¿Crees que este método es suficiente para recomendar fichaje de jugadores? ¿Por qué?